In [ ]:
import scvelo as scv
import scanpy as sc
import pandas as pd
import numpy as np

sc.set_figure_params(scanpy=True, dpi=600)

In [ ]:
NRPC = scv.read("/storage/singlecell/zz4/fetal_bash/results/NRPC_fate/NRPC_fate.h5ad")

In [ ]:
sc.pl.umap(NRPC, color="leiden", legend_loc="on data")

In [ ]:
NRPC.obs['terminal_state_validated'] = ""

In [ ]:
AC = pd.read_csv("/storage/singlecell/zz4/fetal_bash/results/cell_annotation_results/NRPC_AC.csv")
NRPC.obs.loc[NRPC.obs.index.isin(AC["Unnamed: 0"]), "terminal_state_validated"] = "AC"

In [ ]:
BC = pd.read_csv("/storage/singlecell/zz4/fetal_bash/results/cell_annotation_results/NRPC_BC.csv")
NRPC.obs.loc[NRPC.obs.index.isin(BC["Unnamed: 0"]), "terminal_state_validated"] = "BC"

In [ ]:
HC = pd.read_csv("/storage/singlecell/zz4/fetal_bash/results/cell_annotation_results/NRPC_HC.csv")
NRPC.obs.loc[NRPC.obs.index.isin(HC["Unnamed: 0"]), "terminal_state_validated"] = "HC"

In [ ]:
RGC = pd.read_csv("/storage/singlecell/zz4/fetal_bash/results/cell_annotation_results/NRPC_RGC.csv")
NRPC.obs.loc[NRPC.obs.index.isin(RGC["Unnamed: 0"]), "terminal_state_validated"] = "RGC"

In [ ]:
Rod = pd.read_csv("/storage/singlecell/zz4/fetal_bash/results/cell_annotation_results/NRPC_Rod.csv")
NRPC.obs.loc[NRPC.obs.index.isin(Rod["Unnamed: 0"]), "terminal_state_validated"] = "Rod"

In [ ]:
Cone = pd.read_csv("/storage/singlecell/zz4/fetal_bash/results/cell_annotation_results/NRPC_Cone.csv")
NRPC.obs.loc[NRPC.obs.index.isin(Cone["Unnamed: 0"]), "terminal_state_validated"] = "Cone"

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

df = NRPC.obs
df = df.loc[df["terminal_state_validated"] != "", :]
sns.boxplot(
    x=df["terminal_state_validated"],
    y=df["Days"],
    hue=df["Region"],
    order=["RGC", "HC", "Cone", "AC", "Rod", "BC"],
).set(xlabel="NRPC inferred fate", ylabel="Fetal Days")
plt.legend(loc="lower right", bbox_to_anchor=(1.5, 0), title="Region")
plt.savefig(
    "/storage/singlecell/zz4/fetal/figures/NRPC inferred fate_days.svg",
    bbox_inches="tight",
)

In [ ]:
sc.pl.umap(NRPC, color="terminal_state_validated", legend_loc="on data")

In [ ]:
import hotspot
NRPC_ = NRPC[NRPC.obs["terminal_state_validated"] != ""]
sc.pp.calculate_qc_metrics(NRPC_, inplace=True)
NRPC_.obsm["terminal_state_validated"] = pd.get_dummies(
    NRPC_.obs.terminal_state_validated
).to_numpy()

In [ ]:
NRPC_ = NRPC_[:, NRPC_.var.mean_counts > 0]

In [ ]:
NRPC_

In [ ]:
import hotspot
import joblib
hs = joblib.load("/storage/singlecell/zz4/fetal_bash/results/NRPC_hotspot/NPRC_hs.create_modules.pkl")
adata = joblib.load("/storage/singlecell/zz4/fetal_bash/results/NRPC_hotspot/NPRC_hs.adata.pkl")

In [ ]:
modules = hs.create_modules(min_gene_threshold=250, core_only=True, fdr_threshold=0.05)
hs.plot_local_correlations()
plt.savefig('/storage/singlecell/zz4/fetal_bash/results/NRPC_hotspot/plot_local_correlations.svg')

In [ ]:
import scvelo as scv
import plotly.express as px
import scanpy as sc
import hotspot
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import os
from scipy.cluster.hierarchy import leaves_list

module_scores = hs.calculate_module_scores()

adata.obs["Module1"] = module_scores[1].values
adata.obs["Module2"] = module_scores[2].values
adata.obs["Module3"] = module_scores[3].values
adata.obs["Module4"] = module_scores[4].values
adata.obs["Module5"] = module_scores[5].values

module_list = [
    "Module1",
    "Module2",
    "Module3",
    "Module4",
    "Module5",
]
cols = [
    "#636EFA",
    "#EF553B",
    "#00CC96",
    "#AB63FA",
    "#FFA15A",
    "#19D3F3",
    "#FF6692",
    "#B6E880",
    "#FF97FF",
    "#FECB52",
]
for i in range(len(module_list)):
    width = 1000
    height = 1000
    legend_size = 3
    marker_size = 8
    df = adata.obs.copy()
    df["cell_label"] = adata.obs.index.values
    df["x"] = adata.obsm["X_umap"][:, 0]
    df["y"] = adata.obsm["X_umap"][:, 1]
    fig = px.scatter(
        df,
        x="x",
        y="y",
        color=module_list[i],
        width=width,
        height=height,
        color_continuous_scale=["#BAB0AC", cols[i]],
        hover_data=["cell_label"],
        range_color=[2, 5],
    )
    fig.update_traces(mode="markers", marker_size=marker_size)
    fig.update_layout(showlegend=False)
    fig.update_xaxes(showgrid=False)
    fig.update_yaxes(showgrid=False)
    fig.update_layout(yaxis_visible=False, yaxis_showticklabels=False)
    fig.update_layout(xaxis_visible=False, xaxis_showticklabels=False)
    fig.update_layout(
        {
            "plot_bgcolor": "rgba(0, 0, 0, 0)",
            "paper_bgcolor": "rgba(0, 0, 0, 0)",
        }
    )
    fig.update_layout(legend={"title_text": ""})
    fig.update_layout(legend={"itemsizing": "constant"})
    fig.update_layout(legend=dict(font=dict(size=legend_size)))
    fig.write_image(
        "/storage/singlecell/zz4/fetal_bash/results/NRPC_hotspot/NRPC_gene_"
        + module_list[i]
        + ".svg"
    )

In [ ]:
module_scores

In [ ]:
module_scores["terminal_state_validated"] = NRPC_.obs.loc[
    module_scores.index, "terminal_state_validated"
].replace({"BC": "BC/Rod", "Rod": "BC/Rod"})

In [ ]:
module_scores

In [ ]:
module_scores.groupby('terminal_state_validated').mean()

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt

uniform_data = module_scores.groupby("terminal_state_validated").mean()
uniform_data = uniform_data[[3, 1, 5, 2, 4]].transpose()
ax = sns.heatmap(
    uniform_data, linewidth=0.5, cmap="viridis", vmin=-5, vmax=10, annot=True
)
ax.set(xlabel="NRPC inferred fate", ylabel="Gene modules")
plt.savefig(
    "/storage/singlecell/zz4/fetal_bash/results/NRPC_hotspot/module_score.svg",
    bbox_inches="tight",
)

In [ ]:
TFs = pd.read_csv("/storage/singlecell/zz4/fetal_bash/data/TFsDatabaseExtract_v_1.01.csv")

In [ ]:
TFs = TFs.loc[TFs['Is TF?'] == "Yes",]

In [ ]:
NRPC_ = NRPC_[:, NRPC_.var.index.isin(TFs["HGNC symbol"])]

In [ ]:
NRPC_

In [ ]:
sc.pp.normalize_total(NRPC_, target_sum=1e4)
sc.pp.log1p(NRPC_)
sc.tl.rank_genes_groups(NRPC_, groupby="terminal_state_validated")

In [ ]:
sc.pl.rank_genes_groups_dotplot(
    NRPC_,
    n_genes=10,
    save="sc.pl.rank_genes_groups_dotplot.svg",
)